In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [2]:
names = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten', 'eleven',
         'twelve', 'thirteen', 'fourteen', 'fifteen', 'sixteen']
number_of_layers = 12 #np.random.randint(0,15)
neighbors = np.random.randint(1,200)
df =  pd.DataFrame()
    
for j in np.arange(3, 5.5,0.1):
    rolling = pd.DataFrame()
    rolling['zero'] = 3*np.sin(np.arange(0,20,0.1)/6.28)
    for i in range(len(names[0:number_of_layers])):
        layer_elevation = 3*np.sin(np.arange(0,20,0.1)/6.28)+np.random.uniform(0,i)
        elevation = np.full(200, j)
        layer_elevation = np.where(layer_elevation > elevation, elevation, layer_elevation)
        rolling[names[i]] = layer_elevation
    df = pd.concat((df, rolling))
angular_thicknesses = df.T.diff(axis=0).abs().T
neigh = angular_thicknesses.shift(neighbors)
logged = angular_thicknesses.apply(np.log)
powered= angular_thicknesses.apply(lambda x: x**10)
angular_thicknesses = pd.concat([angular_thicknesses, neigh, logged, powered], axis=1, join_axes=[angular_thicknesses.index]
                               ).drop(columns=['zero']).dropna().replace(-np.inf, 0)

#now for onlap section
df_onlap =  pd.DataFrame()
    
for j in np.arange(3, 5.5,0.1):
    rolling = pd.DataFrame()
    rolling['zero'] = 3*np.sin(np.arange(0,20,0.1)/6.28)
    for i in range(len(names[0:number_of_layers])):
        zero = 3*np.sin(np.arange(0,20,0.1)/6.28)
        randomness = np.random.uniform(0,i)
        elevation = np.full(200, j)
        onlap = np.where(np.full(200,randomness) > zero, np.full(200,randomness), zero)
        layer_elevation = np.where( onlap > elevation, elevation,  onlap)
        rolling[names[i]] = layer_elevation
    df_onlap = pd.concat((df, rolling))
onlap_thicknesses = df_onlap.T.diff(axis=0).abs().T
neigh = onlap_thicknesses.shift(neighbors)
logged = onlap_thicknesses.apply(np.log)
powered= onlap_thicknesses.apply(lambda x: x**10)
onlap_thicknesses = pd.concat([onlap_thicknesses, neigh, logged, powered], axis=1, join_axes=[onlap_thicknesses.index]
                               ).drop(columns=['zero']).dropna().replace(-np.inf, 0)

#now for horizontal stratification

df_horizontal =  pd.DataFrame()
    
for j in np.arange(3, 5.5,0.1):
    rolling = pd.DataFrame()
    rolling['zero'] = np.full(200, 0)-np.random.rand(200)/10
    for i in range(len(names[0:number_of_layers])):
        randomness = np.random.uniform(0,i)
        rolling[names[i]] = np.full(200, randomness)+np.random.rand(200)/100
    df_horizontal = pd.concat((df, rolling))
horizontal_thicknesses = df_horizontal.T.diff(axis=0).abs().T
neigh = horizontal_thicknesses.shift(neighbors)
logged = horizontal_thicknesses.apply(np.log)
powered= horizontal_thicknesses.apply(lambda x: x**10)

horizontal_thicknesses = pd.concat([horizontal_thicknesses, neigh, logged, powered], axis=1, join_axes=[horizontal_thicknesses.index]
                               ).drop(columns=['zero']).dropna().replace(-np.inf, 0)

#normalize the data
normalized_df=(angular_thicknesses-angular_thicknesses.min())/(angular_thicknesses.max()-angular_thicknesses.min()).replace(0,0.00001)
normalized_dfo=(onlap_thicknesses-onlap_thicknesses.min())/(onlap_thicknesses.max()-onlap_thicknesses.min()).replace(0,0.00001)
normalized_dfh=(horizontal_thicknesses-horizontal_thicknesses.min())/(horizontal_thicknesses.max()-horizontal_thicknesses.min()).replace(0,0.00001)
normalized_dfo['class'] = 1
normalized_df['class'] = 0
normalized_dfh['class'] = 2

#merge into new dataset
dataset = pd.concat((normalized_df, normalized_dfo, normalized_dfh))


In [3]:
dataset.columns

Index(['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine',
       'ten', 'eleven', 'twelve', 'one', 'two', 'three', 'four', 'five', 'six',
       'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve', 'one', 'two',
       'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten',
       'eleven', 'twelve', 'one', 'two', 'three', 'four', 'five', 'six',
       'seven', 'eight', 'nine', 'ten', 'eleven', 'twelve', 'class'],
      dtype='object')

In [4]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(dataset.iloc[0:, 0:-1].values, dataset.iloc[0:,-1].values, test_size=0.4, random_state=86)
#X_train_rand, X_test_rand, y_train_rand, y_test_rand =train_test_split(randomData[0:,0:7], randomData[0:,7], test_size=0.2, random_state=86)

In [5]:
from keras.utils import to_categorical
y_train_encoded = to_categorical(y_train)
y_test_encoded = to_categorical(y_test)

y_train_rand_encoded = to_categorical(np.random.randint(0,3, 11357))
y_test_rand_encoded = to_categorical(np.random.randint(0,3, 2840))

C:\Users\jesse.pisel\AppData\Local\Continuum\anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [6]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Conv1D

In [7]:
model = Sequential()
model.add(Dense(100, input_shape=(X_train.shape[1],), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(3, kernel_initializer='random_uniform', activation='sigmoid'))

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics=['categorical_accuracy'])

In [ ]:
model.fit(X_train, y_train_encoded, epochs=100, batch_size=32)

Epoch 1/100
9150/9150 [==============================] - 1s 66us/step - loss: 0.6629 - categorical_accuracy: 0.6385
Epoch 2/100
9150/9150 [==============================] - 0s 41us/step - loss: 0.4744 - categorical_accuracy: 0.6749
Epoch 3/100
9150/9150 [==============================] - 0s 39us/step - loss: 0.4670 - categorical_accuracy: 0.6743
Epoch 4/100
9150/9150 [==============================] - 0s 38us/step - loss: 0.4621 - categorical_accuracy: 0.6756
Epoch 5/100
9150/9150 [==============================] - 0s 36us/step - loss: 0.4583 - categorical_accuracy: 0.6813
Epoch 6/100
9150/9150 [==============================] - 0s 36us/step - loss: 0.4563 - categorical_accuracy: 0.6819
Epoch 7/100
9150/9150 [==============================] - 0s 34us/step - loss: 0.4545 - categorical_accuracy: 0.6799
Epoch 8/100
9150/9150 [==============================] - 0s 32us/step - loss: 0.4533 - categorical_accuracy: 0.6836
Epoch 9/100
9150/9150 [==============================] - 0s 33us/step - 

In [ ]:
score=model.evaluate(X_test, y_test_encoded)
score

In [ ]:
preds = model.predict(X_test)

In [ ]:
import scikitplot as skplt
skplt.metrics.plot_confusion_matrix(y_test, np.argmax(preds, axis=1), normalize=True)

In [ ]:
tops = pd.read_csv(r'D:\topsexport05292018.csv')

In [ ]:
tops2 = tops.pivot_table('depth', 'formation', 'api').fillna(0).T
tops2.columns

In [ ]:
tops2 = tops2[['MUDDY', 'MOWRY', 'BELLE_FOURCHE', 'GREENHORN',  'NIOBRARA', 'PARKMAN_BASE', 'PARKMAN_TOP',
               'TECKLA_BASE', 'TECKLA_TOP','TEAPOT_BASE', 'TEAPOT_TOP', 'PIERRE', 'FOX_HILLS' ]]

In [ ]:
tops3 = tops2.diff(axis=1).abs().drop(columns=['MUDDY']).dropna()

In [ ]:
neigh = tops3.shift(10)
logged = tops3.apply(np.log)
powered= tops3.apply(lambda x: x**10)
tops3 = pd.concat([tops3, neigh, logged, powered], axis=1, join_axes=[tops3.index]
                               ).dropna().replace(-np.inf, 0)

In [ ]:
normalized_tops3=(tops3-tops3.min())/(tops3.max()-tops3.min()).replace(0,0.00001)
real_data = normalized_tops3.values

In [ ]:
well_preds = model.predict(real_data)

In [ ]:
predictions = np.argmax(well_preds, axis=1)

In [ ]:
plt.hist(predictions)